In [ ]:
import socketserver
import json
import uuid

from multiprocessing import Pool
import time


class RPCServer(socketserver.BaseRequestHandler):
    def real_solve(self, req):
        if not hasattr(self, req["functionName"]):
            msg = json.dumps({
                "type": "response",
                "result": "illegal function"
            })
            self.request.sendall(msg.encode("utf-8"))
        else:
            result = str(getattr(self, req["functionName"])(req["parameters"]))

            msg = json.dumps({"type": "response", "result": result})
            self.request.sendall(msg.encode("utf-8"))

        print("thread over")

    def handle(self):

        print("conn is :", self.request)  # conn
        print("addr is :", self.client_address)  # addr
        pool = Pool(5)

        while True:
            try:
                #收消息
                print("new ready")
                data = self.request.recv(1024)
                if not data: break
                print("收到客户端的消息是", data.decode("utf-8"))
                req = json.loads(data.decode("utf-8"))
                pool.apply_async(self.real_solve, [
                    req,
                ])
                print("process start")
            except Exception as e:
                print(e)
                break


class MyServer(RPCServer):
    def add(self, parameters):
        time.sleep(10)
        result = 0
        for i in range(len(parameters)):
            result = result + parameters[i]
        return result


if __name__ == "__main__":
    ip_port = ("127.0.0.1", 8000)
    s = socketserver.ThreadingTCPServer(ip_port, MyServer)
    s.serve_forever()

conn is : <socket.socket fd=59, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 8000), raddr=('127.0.0.1', 51707)>
addr is : ('127.0.0.1', 51707)
new ready
收到客户端的消息是 {"type": "request", "async": true, "functionName": "add", "parameters": [1, 2]}
process start
new ready
